# Noxus Client SDK - README Follow Through

In [ ]:
%load_ext autoreload
%autoreload 2

### Client Initialization

In [ ]:
from noxus_sdk.client import Client

# Initialize the client with your API key
client = Client(api_key="your_api_key_here")

#In case you are running a local backend don't forget to provide it
client = Client(api_key="your_api_key_here", base_url="your_noxus_backend_url_here")

### Quick Example

In [ ]:
# List all workflows
workflows = client.workflows.list()
for workflow in workflows:
    print(workflow.name)

# Create a new conversation
from noxus_sdk.resources.conversations import ConversationSettings

settings = ConversationSettings(
    model_selection=["gpt-4o-mini"],
    temperature=0.7,
    max_tokens=150,
    tools=[],
    extra_instructions="Please answer in portuguese."
)

conversation = client.conversations.create(name="New Conversation", settings=settings)
print(conversation.id)

## Workflows

### Building Workflows

In [ ]:
from noxus_sdk.workflows import WorkflowDefinition

# Create a new workflow definition
workflow_def = WorkflowDefinition(name="Simple Workflow")

# Add nodes to the workflow
input_node = workflow_def.node("InputNode").config(label="Fixed Input", fixed_value=True, value="Write a joke.", type="str")
ai_node = workflow_def.node("TextGenerationNode").config(
    template="Please insert a fact about an animal after fulfilling the following request: ((Input 1))",
    model=["gpt-4o-mini"],
)
output_node = workflow_def.node("OutputNode")

# Connect nodes together (from output to input)
workflow_def.link(input_node.output(), ai_node.input("variables", "Input 1"))
workflow_def.link(ai_node.output(), output_node.input())

# Save the workflow to Noxus
simple_workflow = client.workflows.save(workflow_def)
print(f"Created workflow with ID: {simple_workflow.id}")

More complex workflow:

In [ ]:
from noxus_sdk.workflows import WorkflowDefinition

# Create a workflow for summarizing and analyzing text
workflow_def = WorkflowDefinition(name="Complex Workflow")

# Add nodes in sequence
input_node = workflow_def.node("InputNode")
text_generator = workflow_def.node("TextGenerationNode").config(template="Write an essay on ((Input 1)).")
summarizer = workflow_def.node("SummaryNode").config(summary_format="Concise", summary_topic="Summarize the essay in around 300 words.")
compose = workflow_def.node("ComposeTextNode").config(template="Summary: \n\n ((Input 1)) \n\n Extended text: \n ((Input 2))")
output = workflow_def.node("OutputNode")

# Create a branched workflow with multiple paths
workflow_def.link(input_node.output(), text_generator.input("variables", key="Input 1"))
workflow_def.link(text_generator.output(), summarizer.input())
workflow_def.link(summarizer.output(), compose.input("variables", key="Input 1"))
workflow_def.link(text_generator.output(), compose.input("variables", key="Input 2"))
workflow_def.link(compose.output(), output.input())

# Save the workflow
complex_workflow = client.workflows.save(workflow_def)
print(f"Created workflow with ID: {complex_workflow.id}")

### Listing Workflows

In [ ]:
# List all workflows
workflows = client.workflows.list(page=1, page_size=10)
for workflow in workflows:
    print(f"Workflow ID: {workflow.id}, Name: {workflow.name}")

# Get a specific workflow
#workflow = client.workflows.get(workflow_id="your_workfflow_id")
#print(f"Specific Workflow ID: {workflow.id}, Name: {workflow.name}")

### Updating Workflows

In [ ]:
#We fetch the workflow to update
workflow_to_update = client.workflows.get(workflow_id=complex_workflow.id)

# Update existing workflow name
workflow_to_update.name = "Essay Writer"

# Let's add an extra input with the author of the workflow
author_input = workflow_to_update.node("InputNode").config(label="Author", fixed_value=True, value="John Peter Table", type="str")
# We update the compose node config and connection (notice how we use the label)
compose = [w for w in workflow_to_update.nodes if w.type == "ComposeTextNode"][0]
compose.config(
    template="Summary: \n\n ((Input 1)) \n\n Extended text: \n ((Input 2)) \n\n Author: ((Author))",
)
workflow_to_update.link(
    author_input.output("output"), compose.input("variables", "Author")
)

# Update the workflow
updated_workflow = client.workflows.update(workflow_to_update.id, workflow_to_update)
print(f"Updated workflow {updated_workflow.name}")

### Run Workflow

Let's use the work

In [ ]:
workflow = client.workflows.get(workflow_id=simple_workflow.id)

# Run a workflow with no inputs or fixed inputs
run = workflow.run(body={})

# Wait for the workflow to complete
result = run.wait(interval=5)
print(f"Run status: {result.status}")
print(f"Output: {result.output}")

In [ ]:
# Let's override the workflow
run = simple_workflow.run(body={"Fixed Input": "Write a poem"})

# Wait for the workflow to complete
result = run.wait(interval=5)
print(f"Run status: {result.status}")
print(f"Output: {result.output}")

### Listing runs

In [ ]:
# Get a list of workflow runs
runs = client.runs.list(workflow_id=simple_workflow.id, page=1, page_size=10)
for run in runs:
    print(f"Run ID: {run.id}")

# Knowledge Bases

### Creating a Knowledge Base

In [ ]:
from noxus_sdk.resources.knowledge_bases import (
    KnowledgeBaseSettings,
    KnowledgeBaseIngestion,
    KnowledgeBaseRetrieval,
    KnowledgeBaseHybridSettings,
)

# Define knowledge base components
settings = KnowledgeBaseSettings(
    ingestion=KnowledgeBaseIngestion(
        batch_size=10,
        default_chunk_size=1000,
        default_chunk_overlap=200,
        enrich_chunks_mode="contextual",
        enrich_pre_made_qa=False,
    ),
    retrieval=KnowledgeBaseRetrieval(
        type="hybrid_reranking",
        hybrid_settings={"fts_weight": 0.3},
        reranker_settings={}
    ),
)

# Create a new knowledge base
knowledge_base = client.knowledge_bases.create(
    name="Example Knowledge Base",
    description="A sample knowledge base",
    document_types=["pdf", "txt"],
    settings_=settings
)
print(f"Created Knowledge Base ID: {knowledge_base.id}")

### Listing Knowledge Bases

In [ ]:
# List all knowledge bases
knowledge_bases = client.knowledge_bases.list(page=1, page_size=10)
for kb in knowledge_bases:
    print(f"Knowledge Base ID: {kb.id}, Name: {kb.name}")

# Get a specific knowledge base
#kb = client.knowledge_bases.get(knowledge_base_id="your_knowledge_base_id")

### File Operations in Knowledge Base

In [ ]:
from noxus_sdk.resources.knowledge_bases import (
    UpdateDocument,
    DocumentStatus,
    RunStatus,
    File,
    Source,
    DocumentSource,
    DocumentSourceConfig,
    UpdateDocument
)

kb = client.knowledge_bases.get(knowledge_base_id=knowledge_base.id)

#Add a file
run_ids = kb.upload_document(
    files=["notebook_kb_test.txt"],
    prefix="/files" #Where it will be stored on the KB
)
print(f"Upload started with run IDs: {run_ids}")

# We can also monitor runs
print("\n=== Checking Runs ===")
runs = kb.get_runs(status="running")
print(f"Active runs: {len(runs)}")

In [ ]:
# Lets wait for a bit for ingestion to finish
import time
time.sleep(15)

In [ ]:
# List documents
print("\n=== Listing Documents ===")
documents = kb.list_documents(status="trained")
for doc in documents:
    print(f"Document: {doc.name} (Status: {doc.status})")

# Get and update a document if any exist
if documents:
    doc = kb.get_document(documents[0].id)
    print(f"\nGot document: {doc.name}")

    updated_doc = kb.update_document(
        doc.id, 
        UpdateDocument(prefix="/updated/path")
    )
    print(f"Updated document prefix to: {updated_doc.prefix}")

# Refresh KB to see latest status
kb.refresh()
print(f"\nKB status: {kb.status}")
print(f"Total documents: {kb.total_documents}")
print(f"Trained documents: {kb.trained_documents}")
print(f"Error documents: {kb.error_documents}")

# List all knowledge bases
print("\n=== All Knowledge Bases ===")
knowledge_bases = client.knowledge_bases.list(page=1, page_size=10)
for kb in knowledge_bases:
    print(f"KB: {kb.name} (ID: {kb.id})")

# Cleanup
print("\n=== Cleanup ===")
if documents:
    for doc in documents:
        kb.delete_document(doc.id)
        print(f"Deleted document: {doc.name}")

success = kb.delete()
print(f"Knowledge base deletion: {'successful' if success else 'failed'}")

# Conversation

### Creating a Conversation

In [ ]:
from noxus_sdk.resources.conversations import (
    ConversationSettings,
    MessageRequest,
    WebResearchTool,
)

# Create conversation tools
web_research_tool = WebResearchTool(
    enabled=True,
    extra_instructions="Focus on recent and reliable sources."
)

# Define conversation settings
settings = ConversationSettings(
    model_selection=["gpt-4o-mini"],
    temperature=0.7,
    max_tokens=150,
    tools=[web_research_tool],
    extra_instructions="Please be concise."
)

# Create a new conversation
conversation = client.conversations.create(name="Example Conversation", settings=settings)
print(f"Created Conversation ID: {conversation.id}")

### List Conversations

In [ ]:
# List all conversations
conversations = client.conversations.list(page=1, page_size=10)
for conv in conversations:
    print(f"Conversation ID: {conv.id}, Name: {conv.name}")

# Get a specific conversation
#conversation = client.conversations.get(conversation_id="conversation_id_here")

### Sending a Message

Let's use the conversation from above

In [ ]:
import base64
from noxus_sdk.resources.conversations import MessageRequest, ConversationFile

# Simple message without using any tools
message = MessageRequest(content="How are you?")
response = conversation.add_message(message)
print(f"AI Response: {response.message_parts} \n\n")

# Message using web research tool
web_research_message = MessageRequest(
    content="What is the wordle word of yesterday?",
    tool="web_research"
)
response = conversation.add_message(web_research_message)
print(f"Web Research Tool response: {response.message_parts} \n\n")

# Message with attached files
file = ConversationFile(
    name="test.txt",
    status="success",
    b64_content=base64.b64encode(b"Hello, world!").decode("utf-8"),
)
message = MessageRequest(content="What does the file say?", files=[file])
response = conversation.add_message(message)
print(f"Message with file response: {response.message_parts} \n\n")

We can also get all messages in a conversation

In [ ]:
# Get all messages in a conversation
all_messages = conversation.get_messages()
for msg in all_messages:
    print(f"Message ID: {msg.id}, Created: {msg.created_at}")

### Deleting a Conversation

In [ ]:
# Delete a conversation
# client.conversations.delete(conversation_id="conversation_id_here")

### Asynchronous Conversation Operations

In [ ]:
import asyncio

async def conversation_example():
    # Create a conversation asynchronously
    conversation = await client.conversations.acreate(name="Async Example", settings=settings) # Using the settings from the KB section
    
    # Send a message and get response asynchronously
    message = MessageRequest(content="How does quantum computing work?")
    response = await conversation.aadd_message(message)
    
    # Refresh the conversation to get latest state
    updated_conversation = await conversation.arefresh()
    
    # Get all messages
    messages = await updated_conversation.aget_messages()
    return messages


global messages  # Make 'messages' accessible outside the main function
messages = await conversation_example()
print(messages)


### Conversation Tools

Notice that you must provide a `workflow id` and a `knowledge base id` to use the respective tools. 

In [ ]:
from noxus_sdk.resources.conversations import (
    WebResearchTool,
    NoxusQaTool,
    KnowledgeBaseSelectorTool,
    KnowledgeBaseQaTool,
    WorkflowTool
)

# Web research tool
web_tool = WebResearchTool(
    enabled=True,
    extra_instructions="Focus on academic sources"
)

# Noxus Q&A tool
noxus_qa_tool = NoxusQaTool(
    enabled=True,
    extra_instructions="Explain Noxus features in simple terms"
)

# Knowledge base selector tool
kb_selector_tool = KnowledgeBaseSelectorTool(
    enabled=True,
    extra_instructions="Choose the most relevant knowledge base"
)

# Knowledge base Q&A tool with specific KB
kb_qa_tool = KnowledgeBaseQaTool(
    enabled=True,
    kb_id="knowledge_base_uuid_here",
    extra_instructions="Provide detailed answers from the knowledge base"
)

# Workflow execution tool
workflow_tool = WorkflowTool(
    enabled=True,
    workflow={"id": "your_workflow_id", "name": "Workflow Name", "description": "Workflow Description"},
    name="Data Analysis Workflow",
    description="Run the data analysis workflow on provided input"
)

# Create settings with all tools
settings = ConversationSettings(
    model_selection=["gpt-4o-mini"],
    temperature=0.7,
    max_tokens=150,
    tools=[web_tool, noxus_qa_tool, kb_selector_tool, kb_qa_tool, workflow_tool],
    extra_instructions="Use the most appropriate tool for each query."
)

conversation = client.conversations.create(name="Tooljacked Conversation", settings=settings)

# Agents

### Creating an Agent

In [ ]:
from noxus_sdk.resources.conversations import WebResearchTool
from noxus_sdk.resources.conversations import WorkflowTool
from noxus_sdk.resources.conversations import ConversationSettings

# Workflow execution tool
workflow_tool = WorkflowTool(
    enabled=True,
    workflow={"id": simple_workflow.id, "name":"Generate a joke", "description": "Use this tool to generate a joke"},
    name="Simple Workflow",
    description="Just runs"
)


# Define agent settings
agent_settings = ConversationSettings(
    model_selection=["gpt-4o-mini"],
    temperature=0.7,
    max_tokens=150,
    tools=[workflow_tool],
    extra_instructions="Please be helpful and concise."
)

# Create a new agent
agent = client.agents.create(name="Example Agent", settings=agent_settings)
print(f"Created Agent with ID: {agent.id}, and name: {agent.name}")

### List all agents

In [ ]:
agents = client.agents.list()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}")

# Get a specific agent
# agent = client.agents.get(agent_id="your_agent_id")

### Updating and Deleting an Agent

In [ ]:
# Update an agent
updated_agent = client.agents.update(
    agent_id=agent.id,
    name="Updated Agent Name",
    settings=agent_settings
)
print(updated_agent.name)

In [ ]:
# Delete an agent
#client.agents.delete(agent_id="agent_id")

### Starting a conversation with an Agent

In [ ]:
# Get the agent you want to chat with
agent = client.agents.get(agent_id=agent.id)

# Create a conversation with this agent
conversation = client.conversations.create(
    name="Chat with Agent",
    agent_id=agent.id
)

In [ ]:


# Now you can send messages to the conversation
message = MessageRequest(content="Hello, what model are you using?")
response = conversation.add_message(message)
print(f"Agent Response: {response.message_parts}")

In [ ]:
# Let's ask the agent to run the simple workflow
message = MessageRequest(content="Hello, can you run Simple Workflow and tell it to generate a poem?")
response = conversation.add_message(message)
print(f"Agent Response: {response.message_parts}")

You can also create a conversation with the agent asynchronously

In [ ]:
async def create_agent_conversation():
    agent = await client.agents.aget(agent_id="agent_id_here")
    conversation = await client.conversations.acreate(
        name="Async Agent Chat",
        agent_id=agent.id
    )
    return conversation

# Advanced Usage

The SDK supports asynchronous operations.
Here's an example:

In [ ]:
import asyncio
from noxus_sdk.client import Client

async def main():
    # List workflows asynchronously
    workflows = await client.workflows.alist()
    for workflow in workflows:
        print(workflow.name)

    # Create knowledge base asynchronously
    kb = await client.knowledge_bases.acreate(
        name="Async KB",
        description="Created asynchronously",
        document_types=["pdf"],
        settings_=settings # getting the settings from the KB creation above
    )

    # Run a workflow and wait for completion asynchronously
    workflow = await client.workflows.aget(simple_workflow.id)
    run = await workflow.arun(body={})
    result = await run.a_wait(interval=2)
    print(result)

await main()

# Platform Information Methods

The SDK provides methods to retrieve information about the Noxus platform. For example:

In [ ]:
# Get available workflow nodes
nodes = client.get_nodes()  # Synchronous
nodes = await client.aget_nodes()  # Asynchronous

# Get available language models
models = client.get_models()  # Synchronous
models = await client.aget_models()  # Asynchronous

# Get chat model presets
presets = client.get_chat_presets()  # Synchronous
presets = await client.aget_chat_presets()  # Asynchronous

Here is an example that uses the `get_models` method to configure a conversation

In [ ]:
# Get available models and use them in conversation settings
models = client.get_models()
model_names = [model["name"] for model in models]

settings = ConversationSettings(
    model_selection=[model_names[0]],  # Use the first available model
    temperature=0.7,
    max_tokens=150,
    tools=[],
    extra_instructions="Please be concise."
)

# Create a conversation with the retrieved model
conversation = client.conversations.create(
    name="Model-specific Conversation",
    settings=settings
)